In [2]:
using Test

In [3]:
f = readlines("day15.input");

In [4]:
function parse_input(input)
    flows = Dict{String, Int64}()
    edges = Dict{String, Vector{String}}()
    for line in input
        name_re = r"Valve ([A-Z][A-Z]) has flow rate=\d+; tunnels? leads? to valves? .*"
        flow_re = r"Valve [A-Z][A-Z] has flow rate=(\d+); tunnels? leads? to valves? .*"
        edge_re = r"Valve [A-Z][A-Z] has flow rate=\d+; tunnels? leads? to valves? (.*)"
        name = first(match(name_re, line).captures)
        flows[name] = parse(Int64, first(match(flow_re, line).captures))
        edges[name] = [strip(x) for x in (split(first(match(edge_re, line).captures), ","))]
    end
    flows, edges
end

parse_input (generic function with 1 method)

In [5]:
function build_valve_costs(start, budget, flows, edges)
    queue = Vector{String}()
    visited = Dict{String, Bool}()
    dist = Dict{String, Int64}()
    cost = Dict{String, Float64}()
    for name in keys(flows)
        visited[name] = false
    end

    push!(queue, start)
    visited[start] = true;
    dist[start] = 1;
    cost[start] = flows[start]

    while length(queue) > 0
        valve = popfirst!(queue)
        if dist[valve] > (budget + -2)
            break
        end
        for valve2 in edges[valve]
            if !visited[valve2]
                visited[valve2] = true
                dist[valve2] = dist[valve] + 1;
                cost[valve2] = flows[valve2] / (dist[valve2]+1)
                push!(queue, valve2)
            end
        end
    end
    (cost, dist)
end

build_valve_costs (generic function with 1 method)

In [6]:
flows, edges = parse_input(String.(split("Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II", "\n")))

build_valve_costs("AA", 3, flows, edges)

(Dict("BB" => 4.333333333333333, "AA" => 0.0, "DD" => 6.666666666666667, "II" => 0.0), Dict("BB" => 2, "AA" => 1, "DD" => 2, "II" => 2))

In [12]:
function solve_part_1(input, row)
    flows, edges = parse_input(input)

    position = "AA"
    budget = 30
    total_flow = 0
    total_released = 0

    while(budget > 0)
        costs, dist = build_valve_costs(position, budget, flows, edges)

        if all(iszero, values(costs))
            break
        end

        position = reduce((x, y) -> costs[x] ≥ costs[y] ? x : y, keys(costs))
        
        time_taken = dist[position]
        for i=1:time_taken
            budget -= 1
            total_released += total_flow
        end
        
        total_flow += flows[position]
        flows[position] = 0

        println("Moved to $position. Remaining budget is $budget.")
    end
    
    total_released += budget * total_flow
    total_released
end

solve_part_1 (generic function with 1 method)

In [8]:
function solve_part_2(input, limit)
    0
end

solve_part_2 (generic function with 1 method)

In [13]:
@test solve_part_1(String.(split("Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II", "\n")), 10) == 1651


Moved to DD. Remaining budget is 30.
Moved to JJ. Remaining budget is 30.
Moved to BB. Remaining budget is 30.
Moved to HH. Remaining budget is 30.
Moved to EE. Remaining budget is 30.
Moved to CC. Remaining budget is 30.
Test Failed at c:\code\advent-of-code-2022\day16.ipynb:1
  Expression: solve_part_1(String.(split("Valve AA has flow rate=0; tunnels lead to valves DD, II, BB\nValve BB has flow rate=13; tunnels lead to valves CC, AA\nValve CC has flow rate=2; tunnels lead to valves DD, BB\nValve DD has flow rate=20; tunnels lead to valves CC, AA, EE\nValve EE has flow rate=3; tunnels lead to valves FF, DD\nValve FF has flow rate=0; tunnels lead to valves EE, GG\nValve GG has flow rate=0; tunnels lead to valves FF, HH\nValve HH has flow rate=22; tunnel leads to valve GG\nValve II has flow rate=0; tunnels lead to valves AA, JJ\nValve JJ has flow rate=21; tunnel leads to valve II", "\n")), 10) == 1651
   Evaluated: 3593 == 1651


Test.FallbackTestSetException: Test.FallbackTestSetException("There was an error during testing")

In [10]:
@test solve_part_2(String.(split("Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3", "\n")), 20) == 56000011

Test Failed at c:\code\advent-of-code-2022\day16.ipynb:1
  Expression: solve_part_2(String.(split("Sensor at x=2, y=18: closest beacon is at x=-2, y=15\nSensor at x=9, y=16: closest beacon is at x=10, y=16\nSensor at x=13, y=2: closest beacon is at x=15, y=3\nSensor at x=12, y=14: closest beacon is at x=10, y=16\nSensor at x=10, y=20: closest beacon is at x=10, y=16\nSensor at x=14, y=17: closest beacon is at x=10, y=16\nSensor at x=8, y=7: closest beacon is at x=2, y=10\nSensor at x=2, y=0: closest beacon is at x=2, y=10\nSensor at x=0, y=11: closest beacon is at x=2, y=10\nSensor at x=20, y=14: closest beacon is at x=25, y=17\nSensor at x=17, y=20: closest beacon is at x=21, y=22\nSensor at x=16, y=7: closest beacon is at x=15, y=3\nSensor at x=14, y=3: closest beacon is at x=15, y=3\nSensor at x=20, y=1: closest beacon is at x=15, y=3", "\n")), 20) == 56000011
   Evaluated: 0 == 56000011


Test.FallbackTestSetException: Test.FallbackTestSetException("There was an error during testing")

In [11]:
(solve_part_1(f, 2000000), solve_part_2(f, 4000000))

ErrorException: type Nothing has no field captures